In [ ]:
import sys
import hashlib
import hmac
import json
import urllib
import urllib.parse
import logging
from urllib.request import urlopen, Request
from datetime import datetime


BASE_URL       = "https://openapi.tuyaus.com"
LOGIN_URL      = "/v1.0/token?grant_type=1"
ATTRIBUTES_URL = "/v2.0/cloud/thing/{device_id}/shadow/properties"


def make_request(url, params=None, headers=None):
    if params:
        url = url + "?" + urllib.parse.urlencode(params)
    request = Request(url, headers=headers or {})

    try:
        with urlopen(request, timeout=10) as response:
            return response, response.read().decode("utf-8")

    except Exception as error:
        return error, ""

def get_timestamp(now = datetime.now()):
    return str(int(datetime.timestamp(now)*1000))

def get_sign(payload, key):
    byte_key = bytes(key, 'UTF-8')
    message = payload.encode()
    sign = hmac.new(byte_key, message, hashlib.sha256).hexdigest()
    return sign.upper()

def get_access_token():
    now = datetime.now()

    timestamp = get_timestamp(now)
    string_to_sign = client_id + timestamp + "GET\n" + \
        "e3b0c44298fc1c149afbf4c8996fb92427ae41e4649b934ca495991b7852b855\n" + \
        "\n" + \
        LOGIN_URL
    signed_string = get_sign(string_to_sign, client_secret)

    headers = {
            "client_id": client_id,
            "sign": signed_string,
            "t": timestamp,
            "mode": "cors",
            "sign_method": "HMAC-SHA256",
            "Content-Type": "application/json"
            }

    response, body = make_request(BASE_URL + LOGIN_URL, headers = headers)

    json_result = json.loads(body)["result"]
    access_token = json_result["access_token"]
    return access_token




client_id     = "tuya_client_id"
client_secret = "tuya_client_secret"
access_token  = get_access_token()

In [5]:
import requests

# Set up the endpoint and headers based on the cURL command
url = "https://openapi.tuyaeu.com/v2.0/cloud/thing/bfe22383bb69caf809simm/shadow/properties"
headers = {
    "sign_method": "HMAC-SHA256",
    "client_id": "vtj3ft7hadfea5y3yvs7",
    "t": "1699225656039",
    "mode": "cors",
    "Content-Type": "application/json",
    "sign": "66621B9CF6EBB6A6D1A2DCDD63432D951F613E9B8D35AFA90632F3B6FB223A8F",
    "access_token": "6a7f11dab51faecae86abc551626dc31"
}

# Make the GET request
response = requests.get(url, headers=headers)

# Check if the request was successful
if response.status_code == 200:
    # Parse the response JSON if needed
    data = response.json()
    print(data)
else:
    print("Error:", response.status_code, response.text)



{'result': {'properties': [{'code': 'mode', 'custom_name': '', 'dp_id': 1, 'time': 1699037296012, 'value': 'manual'}, {'code': 'temp_set', 'custom_name': '', 'dp_id': 2, 'time': 1699037295704, 'value': 22}, {'code': 'temp_current', 'custom_name': '', 'dp_id': 3, 'time': 1699224858539, 'value': 180}, {'code': 'switch_rapid', 'custom_name': '', 'dp_id': 4, 'time': 1699201465879, 'value': False}, {'code': 'countdown_rapid', 'custom_name': '', 'dp_id': 5, 'time': 1699037025013, 'value': 0}, {'code': 'work_state', 'custom_name': '', 'dp_id': 7, 'time': 1698890747737, 'value': 'opened'}, {'code': 'window_check', 'custom_name': '', 'dp_id': 8, 'time': 1698883961662, 'value': False}, {'code': 'window_state', 'custom_name': '', 'dp_id': 9, 'time': 1699201465709, 'value': 'closed'}, {'code': 'child_lock', 'custom_name': '', 'dp_id': 13, 'time': 1698883960500, 'value': False}, {'code': 'battery_percentage', 'custom_name': '', 'dp_id': 14, 'time': 1699214838310, 'value': 69}, {'code': 'week_progra